In [205]:
END = 150

# likely not necessary
vitA_reqs = [1000, 1000, 1000, 1000, 1000]
vitD_reqs = [10, 10, 10, 5, 5]
vitE_reqs = [10, 10, 10, 10, 10]
vitK_reqs = [45]

MAX_MEALS = 3

In [206]:
class Meal:
    
    default_id = 0

    def __init__(self, calories: int, protein: int, carbs: int, fats: int, cholesterol: int, sugar: int, name=""):
        
        if name == "":
            self.name = str(Meal.default_id)
            Meal.default_id += 1
        else:
            self.name = name
            
        self.calories = calories
        self.protein = protein
        self.carbs = carbs
        self.fats = fats
        self.cholesterol = cholesterol
        self.sugar = sugar
        
        self.nutrients = locals()
        self.nutrients.pop('self')
        
    def __repr__(self):
        return self.name
        
    def getNutrients(self) -> dict:
        return self.nutrients
    
    def getNutrient(self, nutrient: str) -> int:
        return self.nutrients[nutrient]

In [207]:
m = Meal(10, 10, 10, 10, 10, 10)
m.getNutrients()
m.getNutrient('sugar')

10

In [208]:
class DailyDiet:
    
    def __init__(self, meals=[]):
        self.validateMeals(meals)
        
        self.meals = meals
        
        self.nutrients = self.calculateTotals(meals)
                    
    def validateMeals(self, meals):
        if len(meals) > MAX_MEALS:
            raise ValueError('Too many meals included ')
            
    def calculateTotals(self, meals=None):
        if meals == None:
            meals = self.meals
        nutrients = dict()
        for meal in meals:
            for nutrient, value in meal.getNutrients().items():
                if nutrient not in nutrients:
                    nutrients.update({nutrient:value})
                else:
                    nutrients[nutrient] += value
    
        return nutrients

In [209]:
class Eater:
    
    # age groups
    age_groups = [(11,14), (15,18), (19,24), (25,50), (51,END)]

    # things we want
    calorie_reqs = [1600, 1800, 2000, 1800, 1600]
    protein_reqs = [0.20*item for item in calorie_reqs]
    carb_reqs = [0.45*item for item in calorie_reqs]
    fat_reqs = [0.35*item for item in calorie_reqs]
    
    requirements = [calorie_reqs, protein_reqs, carb_reqs, fat_reqs]

    # things to avoid
    chol_limits = [170, 170, 170, 200, 200]
    sugar_limits = [25, 25, 25, 25, 25]
    
    limits = [chol_limits, sugar_limits]
    
    def __init__(self, age, sex, meals = []):
        self.age = age
        self.age_index = self.findAgeIndex(age)
        self.sex = sex
        self.diet = DailyDiet(meals)
        self.nutrients = self.diet.calculateTotals()
        self.baseline = self.getBaselineNutrients()
        
    def getBaselineNutrients(self):
        i = self.age_index
        baseline = {
            "calories": Eater.calorie_reqs[i],
            "protein": Eater.protein_reqs[i],
            "carbs": Eater.carb_reqs[i],
            "fats": Eater.fat_reqs[i],
            "cholesterol": Eater.chol_limits[i],
            "sugar": Eater.sugar_limits[i]
        }
        return baseline
        
    def findAgeIndex(self, age):
        age_groups = Eater.age_groups
        for i in range(len(age_groups)):
            age_group = age_groups[i]
            if self.age <= age_group[1] and self.age >= age_group[0]:
                return i
            
    def calculateTotals(self):
        return self.diet.calculateTotals()
    
    def calculateDeficiency(self):
        deficiency = {}
        for nutrient, value in self.baseline.items():
            deficiency.update({nutrient: value - self.nutrients[nutrient]})
        return deficiency
    
    def calculateRelativeDeficiency(self, nutrients=None, baseline=None):
        if nutrients == None:
            nutrients = self.nutrients
        if baseline == None:
            baseline = self.baseline
        deficiency = {}
        for nutrient, value in baseline.items():
            deficiency.update({nutrient: 1.00 - (value - nutrients[nutrient])/value})
        return deficiency
        
        
        


In [210]:
m1 = Meal(10, 10, 10, 10, 10, 10)
m2 = Meal(20, 20, 20, 20, 20, 30)
m3 = Meal(1, 2, 3, 4, 5, 6)
m4 = Meal(1000, 40, 200, 40, 250, 100)

meals = [m1, m2, m3]
person = Eater(89, 'M', meals)
print(person.calculateTotals())
print(person.calculateDeficiency())
print(person.calculateRelativeDeficiency())

{'calories': 31, 'protein': 32, 'carbs': 33, 'fats': 34, 'cholesterol': 35, 'sugar': 46, 'name': ''}
{'calories': 1569, 'protein': 288.0, 'carbs': 687.0, 'fats': 526.0, 'cholesterol': 165, 'sugar': -21}
{'calories': 0.01937500000000003, 'protein': 0.09999999999999998, 'carbs': 0.04583333333333328, 'fats': 0.06071428571428572, 'cholesterol': 0.17500000000000004, 'sugar': 1.8399999999999999}


In [211]:
class ScoringEngine:
    
    def __init__(self):
        pass
        
    def calculateNewMealValue(self, eater, new_meal):
        new_nutrients = new_meal.getNutrients()
        deficiency_nutrients = eater.calculateDeficiency()
        completion = eater.calculateRelativeDeficiency(new_nutrients, deficiency_nutrients)
        return completion
        
    def scoreMeal(self, eater, new_meal):
        coverage = self.calculateNewMealValue(eater, new_meal)
        score = coverage['calories']*0.5 + coverage['protein']*0.20 + coverage['carbs']*0.10 + coverage['fats']*0.10 + coverage['cholesterol']*0.05 + coverage['sugar']*0.05
        return score
    
    def scoreMeals(self, eater, new_meals=[]):
        meal_list = sorted(new_meals, key=lambda x: self.scoreMeal(eater, x), reverse=True)
        Meal.default_id = 0
        return meal_list
        

In [212]:
se = ScoringEngine()
print(se.scoreMeals(person, [m1, m2, m3, m4]))

[4, 1, 3, 2]


In [215]:
import requests

In [231]:
content = requests.get("https://api.nal.usda.gov/fdc/v1/foods/list?dataType=Foundation&api_key=D24ch0xJUrhATuYsayVFDFNzBfv8UFMwyYJxVbuN&pageSize=100")

In [248]:
food_ids = ",".join([str(element['fdcId']) for element in content.json()[:20]])
final_url = f'https://api.nal.usda.gov/fdc/v1/foods?fdcIds={food_ids}&api_key=D24ch0xJUrhATuYsayVFDFNzBfv8UFMwyYJxVbuN'
food_content = requests.get(final_url)

In [247]:
print(food_content)

<Response [504]>
